In [2]:
import pandas as pd 
import numpy as np
import seaborn as sns
import pickle 

import math
import torch

import os

import sklearn
import imblearn as imb
# print("imblearn version: ",imblearn.__version__)
from imblearn.over_sampling import SMOTE, ADASYN

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import itertools

from scipy.stats import randint

from rdkit import Chem
import sys
sys.path.append('../../../')
from scipy.spatial import distance


In [3]:
# split_path = '../../../../../data/NEK_data_4Berkeley/NEK2/atom_data'
# train_x_df = pd.read_csv(split_path+"/NEK2_binding_random_fold1_trainX.csv")
# train_y_df= pd.read_csv(split_path+"/NEK2_binding_random_fold1_trainY.csv")
# test_x_df= pd.read_csv(split_path+"/NEK2_binding_random_fold1_testX.csv")
# test_y_df= pd.read_csv(split_path+"/NEK2_binding_random_fold1_testY.csv")
# # train_x = torch.from_numpy(train_x_df.to_numpy())
# # train_y = torch.from_numpy(train_y_df.to_numpy().reshape(-1))
# # test_x = torch.from_numpy(test_x_df.to_numpy())
# # test_y = torch.from_numpy(test_y_df.to_numpy().reshape(-1))
# x_df= pd.concat([train_x_df, test_x_df])
# scaling=StandardScaler()

# scaling.fit(x_df)
# Scaled_data=scaling.transform(x_df)
# train_x_scaledtemp = scaling.transform(train_x_df)
# test_x_scaledtemp = scaling.transform(test_x_df) 

# train_x_df = train_x_scaledtemp
# # train_y_df = pd.DataFrame(train_y)
# test_x_df = test_x_scaledtemp
# train_x_df.shape, train_y_df.shape, test_x_df.shape, test_y_df.shape


In [ ]:
def scale_df(data_path): 
    """Standard Scalar on full dataframe. Pass in full (absolute) datapath"""
    df=pd.read_csv(data_path)
    train_df = df[df['fold']!='fold1']
    test_df = df[df['fold']=='fold1']
    print(f'train_df shape: {train_df.shape}, test_df shape: {test_df.shape}')

    label_col = df['active']
    all_feat_names = df.columns # contains identification compound id and base_rdkit_smiles 
    all_feat_names = all_feat_names.to_list()
    
    id_cols = all_feat_names[:2] # ['compound_id', 'base_rdkit_smiles']
    
    feat_names = all_feat_names[3:] # numerical feat_names
    
    feat_names = feat_names[:-1] # removing fold 
    feat_names = all_feat_names[3:-1] 
    df_numeric = df[feat_names]
    df_id = df[id_cols]
    scaler = StandardScaler()
    df_numeric_scaled = scaler.fit_transform(df_numeric)
    
    df_numeric_scaled = pd.DataFrame(df_numeric_scaled, columns=feat_names)
    
    df_scaled = pd.concat([df_id, label_col, df_numeric_scaled, df[['fold']]], axis=1)
    df_scaled = df_scaled.drop(columns='fold')
    print(f'shape: {df_scaled.shape}')
    return(df_scaled)

        

In [4]:
df= pd.read_csv('/Users/jayceepang/msse/capstone/data/NEK_data_4Berkeley/NEK2/atom_data/NEK2_1_uM_min_50_pct_binding_5fold_random_imbalanced.csv')
train_df = df[df['fold']!='fold1']
test_df = df[df['fold']=='fold1']
print(f'train_df shape: {train_df.shape}, test_df shape: {test_df.shape}')

label_col = df['active']
all_feat_names = df.columns # contains identification compound id and base_rdkit_smiles 
all_feat_names = all_feat_names.to_list()

id_cols = all_feat_names[:2] # ['compound_id', 'base_rdkit_smiles']

feat_names = all_feat_names[3:] # numerical feat_names

feat_names = feat_names[:-1] # removing fold 
feat_names = all_feat_names[3:-1] 
df_numeric = df[feat_names]
df_id = df[id_cols]


scaler = StandardScaler()
df_numeric_scaled = scaler.fit_transform(df_numeric)

df_numeric_scaled = pd.DataFrame(df_numeric_scaled, columns=feat_names)

df_scaled = pd.concat([df_id, label_col, df_numeric_scaled, df[['fold']]], axis=1)

df_scaled = df_scaled.drop(columns='fold')
print(df_scaled.shape)
df_scaled

train_df shape: (1125, 310), test_df shape: (283, 310)
(1408, 309)


,compound_id,base_rdkit_smiles,active,ASA+_per_atom,ASA-,ASA_H_per_atom,ASA_P,ASA_per_atom,BCUT_PEOE_0,BCUT_PEOE_1,...,vsurf_Wp2_per_atom,vsurf_Wp3,vsurf_Wp4,vsurf_Wp5,vsurf_Wp6,vsurf_Wp7,vsurf_Wp8,weinerPath,weinerPol_per_atom,zagreb_per_atom
0,kdb_2562,Cn1cnc2c(N)ncnc21,0,6.171916,-2.880666,-0.242820,-0.138540,3.174018,1.664387,2.956607,...,3.468620,-0.868155,-0.143591,-0.065674,-0.058947,-0.183657,-0.206969,-1.235610,-0.816892,0.194164
1,kdb_2536,FC(F)(F)c1ccc(/C=C/c2cncnc2Nc2ccc3[nH]c(Cc4ccc...,0,-1.253186,1.116825,-0.573082,0.453648,-0.469361,1.194922,-0.908293,...,-0.118570,-0.071295,-0.143591,-0.093917,-0.329423,-0.398969,-0.206969,0.619864,0.096361,0.795410
2,kdb_3056,CNCc1ccc(-c2cc(-c3nc(-c4ccc(S(=O)(=O)C(C)C)cc4...,0,0.504051,0.725849,-0.489283,1.847024,0.319364,-0.163025,-0.395619,...,0.992820,1.502885,0.973047,0.809867,0.211529,-0.291313,-0.206969,0.257966,-0.011573,-0.539664
3,kdb_3510,CNC(=O)Nc1ccc2c(c1)CC[C@@]21OC(=O)N(CC(=O)N(Cc...,0,-1.005664,1.857896,-0.732021,1.562579,-0.222465,-0.258641,0.555600,...,-1.184218,-0.544634,-0.283901,-0.277498,-0.445342,-0.398969,-0.206969,0.769733,1.066145,0.502085
4,kdb_3024,CCCNc1ccc2ncc(-c3ccnc(C)c3)n2n1,0,1.103785,-1.577423,0.249787,-0.935318,-0.015979,0.123578,0.911352,...,-0.463032,-0.850785,-0.482674,-0.630539,-0.986294,-0.398969,-0.206969,-0.941408,-1.388487,-1.295085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1403,kdb_2740,COc1cc(Nc2nccc(-c3c(-c4cccc(NC(=O)c5c(F)cccc5F...,1,-0.971077,0.595670,0.042390,-1.381681,-1.306781,1.076353,-0.770900,...,-1.103084,-0.145118,-0.336518,-0.574052,-0.368063,-0.076001,-0.206969,2.012534,1.458419,0.765553
1404,kdb_2211,C[C@@H](Oc1cc(-n2cnc3ccc(OC4CCN(C)CC4)cc32)sc1...,1,-0.305347,-0.057993,-0.633830,0.219370,-0.942076,-1.316749,0.720930,...,-0.443942,0.501924,0.195493,-0.093917,-0.483981,-0.291313,-0.206969,0.415072,-0.364379,-0.438959
1405,kdb_2887,Cc1ccc(CC(=O)N2CCc3cc(-c4cn(C)c5ncnc(N)c45)ccc...,1,-0.193253,-0.814432,-0.557942,-0.425094,-0.973193,0.467603,0.020012,...,-0.885260,-0.486009,-0.389134,-0.489322,-0.599899,-0.398969,-0.206969,-0.175035,0.233279,0.033356
1406,kdb_2768,CNC(=O)c1ccccc1Nc1nc(Nc2ccc(N3CCOCC3)cc2)ncc1Cl,1,-0.073978,-0.955461,-0.432754,-0.659996,-1.040959,-0.678455,-0.542517,...,-0.275996,0.656085,0.844429,0.739259,0.520645,-0.291313,-0.206969,0.012301,-0.364379,-0.601298


In [5]:
# df_original = pd.read_csv('/Users/jayceepang/msse/capstone/data/NEK_data_4Berkeley/NEK2/atom_data/NEK2_1_uM_min_50_pct_binding_5fold_random_imbalanced.csv')
# df_original
# all_feat_names = df_original.columns # contains identification compound id and base_rdkit_smiles 
# all_feat_names = all_feat_names.to_list()
# id_cols = all_feat_names[:2]
# feat_names = all_feat_names[3:] # numerical feat_names
# feat_names = feat_names[:-1] # removing fold 


In [6]:
def match_id_cols(original_df, modified_df, id_cols):
    """
    Matches rows of the modified dataframe to the original dataframe based on numerical data points,
    and restores identification columns from the original dataframe.

    Parameters:
    original_df (pd.DataFrame): Original dataframe with identification columns.
    modified_df (pd.DataFrame): Modified dataframe with only numerical columns.
    id_cols (list): List of identification columns to restore.

    Returns:
    pd.DataFrame: A new dataframe with identification columns restored.
    """
    numerical_cols = [col for col in modified_df.columns if col in original_df.columns and col not in id_cols]
    id_data = []
    for idx, row in modified_df.iterrows():
        distances = original_df[numerical_cols].apply(lambda x: distance.euclidean(x,row[numerical_cols]), axis=1)

        closest_match_idx = distances.idxmin()
        matched_id = original_df.loc[closest_match_idx, id_cols].values.flatten()
        id_data.append(matched_id)
    
    id_df = pd.DataFrame(id_data,columns=id_cols)
    restored_df = pd.concat([id_df, modified_df.reset_index(drop=True)], axis=1)
    
    return restored_df

In [16]:
data_path = '/Users/jayceepang/msse/capstone/data/NEK_data_4Berkeley/NEK2/bind/'
train_x_temp = pd.read_csv(data_path+'bind_train_x_UNDER.csv',header=None)
train_y_temp = pd.read_csv(data_path+'bind_train_y_UNDER.csv',header=None)
test_x_temp = pd.read_csv(data_path+'bind_test_x_UNDER.csv',header=None)
test_y_temp = pd.read_csv(data_path+'bind_test_y_UNDER.csv',header=None)
print(f'train x: {train_x_temp.shape}, train y: {train_y_temp.shape}, testx: {test_x_temp.shape}, test y shape: {test_y_temp.shape}')


train x: (90, 306), train y: (90, 1), testx: (283, 306), test y shape: (283, 1)


In [9]:
train_x = pd.DataFrame(train_x_temp)
train_x.columns=feat_names
test_x = pd.DataFrame(test_x_temp)
test_x.columns=feat_names
print(test_x.shape, train_x.shape)
train_x['active']=train_y_temp
test_x['active']=test_y_temp

(283, 306) (90, 306)


In [18]:
under_train = match_id_cols(df_scaled, train_x, id_cols)
under_test = match_id_cols(df_scaled, test_x, id_cols)

In [11]:
under_train['subset'] = 'train'
under_test['subset'] = 'test'

In [14]:
under_restored = pd.concat([under_train, under_test])
under_restored
under_restored.to_csv('nek2_moe_bind_UNDER_matched.csv')

In [17]:
under_restored

,compound_id,base_rdkit_smiles,ASA+_per_atom,ASA-,ASA_H_per_atom,ASA_P,ASA_per_atom,BCUT_PEOE_0,BCUT_PEOE_1,BCUT_PEOE_2,...,vsurf_Wp4,vsurf_Wp5,vsurf_Wp6,vsurf_Wp7,vsurf_Wp8,weinerPath,weinerPol_per_atom,zagreb_per_atom,active,subset
0,kdb_2125,Nc1ncc2c(n1)-c1c([nH]c3ccc(C#CCCO)cc13)CCC2,1.835639,-0.968780,0.039665,0.569270,0.903505,0.122121,-1.257673,0.624836,...,1.212744,1.233516,0.829760,-0.291313,-0.206969,-0.740874,0.540647,-0.195450,0,train
1,kdb_196,C=CC(=O)N1CCC[C@@H](n2nc(-c3ccc(Oc4ccccc4)cc3)...,-0.042725,0.341766,-0.004217,0.039535,-0.259640,-1.077339,-0.913236,0.781102,...,-0.237131,-0.235133,-0.213505,-0.291313,-0.206969,0.131514,0.254849,-0.110863,0,train
2,kdb_66,C[C@H]1CNCCCN1S(=O)(=O)c1cccc2cncc(F)c12,-0.188457,-1.195403,-0.163621,-1.030664,-0.636011,-0.644991,0.330950,-0.350738,...,-0.050050,-0.065674,-0.561260,-0.398969,-0.206969,-0.895426,0.739311,-0.861833,0,train
3,kdb_2924,Cc1ccn2c(-c3ccncc3)c(-c3ccc(F)cc3)nc2n1,0.692491,-0.370706,2.075239,-1.694082,1.088615,1.359727,-0.559923,0.932187,...,-1.552541,-1.477836,-1.218131,-0.398969,-0.206969,-0.822620,0.766903,1.046445,0,train
4,kdb_2141,COc1cccc(CCNc2nc3cc(C(=O)O)ccc3c3cncnc23)c1,-0.149779,-0.694584,-0.832352,0.235119,-0.407997,0.234868,-1.224741,0.106698,...,-0.967915,-0.983579,-0.947655,-0.398969,-0.206969,-0.370886,0.721652,0.398712,0,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,kdb_1627,CN[C@H]1C[C@@H]2O[C@](C)([C@H]1OC)n1c3ccccc3c3...,-0.829305,-0.606676,-0.711927,-0.580871,-1.457042,-1.209951,1.095581,-0.537771,...,0.066875,0.132029,-0.058947,-0.398969,-0.206969,-0.128627,3.562265,1.194907,1,test
279,kdb_2758,c1ccc(-c2ccnc3[nH]ccc23)cc1,1.325649,-1.222164,2.374258,-1.635889,1.987426,1.693552,-0.893856,0.765541,...,-0.798373,-0.785877,-0.754457,-0.398969,-0.206969,-1.147903,-0.762591,0.125982,1,test
280,kdb_2288,COCCOCCOc1cc2ncc3c4ccc(C#N)cc4[nH]c3c2cc1OC,1.130839,-0.968577,0.678757,-0.691671,0.130626,0.118424,0.656058,-1.451143,...,-0.745756,-0.715268,-0.483981,-0.398969,-0.206969,-0.236771,0.214837,-0.364932,1,test
281,kdb_2768,CNC(=O)c1ccccc1Nc1nc(Nc2ccc(N3CCOCC3)cc2)ncc1Cl,-0.073978,-0.955461,-0.432754,-0.659996,-1.040959,-0.678455,-0.542517,0.329226,...,0.844429,0.739259,0.520645,-0.291313,-0.206969,0.012301,-0.364379,-0.601298,1,test
